In [33]:
%matplotlib inline
import math as m
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
from SAMPEX_functions import read_counts as read
from SAMPEX_functions import mb_finder, mb_magnitude, iso_calculator, OrbAtt_augment, OrbAtt_augment_loop
import pandas as pd
import os
import pathlib
import string

In [34]:
# defining file
year = '1993'
start_day = '1'
end_day = '365'
Output_Plots = False

In [35]:
# code is specific for year 1993
OrbAtt_names = np.array(['OrbAtt_secofyear_1993001-1993012.txt', 'OrbAtt_secofyear_1993013-1993039.txt', 'OrbAtt_secofyear_1993040-1993066.txt', \
                         'OrbAtt_secofyear_1993067-1993093.txt', 'OrbAtt_secofyear_1993094-1993120.txt', 'OrbAtt_secofyear_1993121-1993147.txt', \
                         'OrbAtt_secofyear_1993148-1993174.txt', 'OrbAtt_secofyear_1993175-1993201.txt', 'OrbAtt_secofyear_1993202-1993228.txt', \
                         'OrbAtt_secofyear_1993229-1993255.txt', 'OrbAtt_secofyear_1993256-1993282.txt', 'OrbAtt_secofyear_1993283-1993309.txt', \
                         'OrbAtt_secofyear_1993310-1993336.txt', 'OrbAtt_secofyear_1993337-1993363.txt', 'OrbAtt_secofyear_1993364-1993365.txt'])

In [36]:
########### INITIALIZING MB & BG ISO vs PARAM HISTOGRAMS ############
max_iso = 1.0
bin_num = 25

PA_bins = np.linspace(0, 180, num=bin_num+1)
MLT_bins = np.linspace(0, 24, num=bin_num+1)

###### MICROBURST ISO vs PARAMETERS #######
# Create empty microburst histograms

H_PAvMLT_MB, _, _ = np.histogram2d(np.array([]), np.array([]), bins=[MLT_bins, PA_bins])

###### BACKGROUND ISO vs PARAMETERS #######
# Create empty background histograms

H_PAvMLT_BG, _, _ = np.histogram2d(np.array([]), np.array([]), bins=[MLT_bins, PA_bins])

In [37]:
for day_of_year in np.arange(int(start_day), int(end_day)+1):
    # name the day
    if len(str(day_of_year)) == 1:
        file = 'hhrr' + year + '00' + str(day_of_year) + '.txt'
    elif len(str(day_of_year)) == 2:
        file = 'hhrr' + year + '0' + str(day_of_year) + '.txt'
    elif len(str(day_of_year)) == 3:
        file = 'hhrr' + year + str(day_of_year) + '.txt'

    # reading in electron counts files:
    t_electrons, r1, r2, r3, r4 = read(year + '_data\\' + file)

    # reading in orbit/attitude data file
    OrbAtt_mask = []
    for i in np.arange(len(OrbAtt_names)):
        OrbAtt_mask.append(day_of_year >= int(OrbAtt_names[i][21:24]) and day_of_year <= int(OrbAtt_names[i][29:32]))
        
    OA_file = OrbAtt_names[OrbAtt_mask][0]
    
    directory = 'E:\SAMPEX_Data\\' + year + '_data\\'
    OrbAtt_file = pd.read_csv(directory + OA_file, names = ['day', 'hr', 'min', 'sec', 'GEO_Radius', 'GEO_Long', 'GEO_Lat', 'Altitude', 
                                                            'L_Shell', 'MLT', 'SAA_Flag', 'Pitch', 'zenith', 'azimuth', 'Att_Flag'], sep = '\s+', header = 70)
    
    # Augment OrbAtt data to fit counts data
    t_OrbAtt, LS_OrbAtt, MLT_OrbAtt, P_OrbAtt, Lat_OrbAtt, Long_OrbAtt, R_OrbAtt, t_electrons, r1, r2, r3, r4 = OrbAtt_augment_loop(t_electrons, r1, r2, r3, r4, OrbAtt_file, day_of_year)   
    
    # find microburst times and N_100, SSD1, SSD4 counts using algorithm
    t_microburst, N_100_microburst, r1_microburst, r4_microburst, mb_index, MB_mask, N_100, A_500 = mb_finder(t_electrons, r1, r2, r3, r4)

    # create background counts mask
    BG_mask = ~MB_mask

    # calculation of the isotropy indices of electron counts
    iso_indices_MB = iso_calculator(r1[MB_mask], r4[MB_mask])
    iso_indices_BG = iso_calculator(r1[BG_mask], r4[BG_mask])

    # find the OrbAtt microburst parameters
    if len(mb_index) > 0:
        t_OrbAtt_mb = t_OrbAtt[mb_index]; LS_OrbAtt_mb = LS_OrbAtt[mb_index]; MLT_OrbAtt_mb = MLT_OrbAtt[mb_index];
        P_OrbAtt_mb = P_OrbAtt[mb_index]; Lat_OrbAtt_mb = Lat_OrbAtt[mb_index]; Long_OrbAtt_mb = Long_OrbAtt[mb_index];
        R_OrbAtt_mb = R_OrbAtt[mb_index]

    # calculation of B_3 bin percentiles
    y_microburst, B_3_microburst, B_3 = mb_magnitude(N_100, N_100_microburst, A_500)
    
    y = N_100 - B_3
    y_background = y[BG_mask]   
    
    ###### MICROBURST ISO vs PARAMETERS #######
    # Create histograms for day #
    if len(mb_index) > 0:

        H_PAvMLT_MB_day, _, _ = np.histogram2d(MLT_OrbAtt_mb, P_OrbAtt_mb, bins=[MLT_bins, PA_bins])
    
        # Add histograms for day # to cumulative histograms
        H_PAvMLT_MB += H_PAvMLT_MB_day
    
    ###### BACKGROUND ISO vs PARAMETERS #######
    # Create histograms for day #
    H_PAvMLT_BG_day, _, _ = np.histogram2d(MLT_OrbAtt[BG_mask], P_OrbAtt[BG_mask], bins=[MLT_bins, PA_bins])
    
    # Add histograms for day # to cumulative histograms
    H_PAvMLT_BG += H_PAvMLT_BG_day

In [1]:
plt.figure(figsize=(7, 10))
plt.title(f'Spacecraft Pitch Angle vs MLT (Microbursts)')
p_PAvMLT_MB = plt.pcolormesh(MLT_bins, PA_bins, H_PAvMLT_MB.T, vmin=0)
plt.colorbar(p_PAvMLT_MB, orientation='horizontal', label='Number of microbursts')
plt.xlabel('MLT')
plt.ylabel('Pitch Angle')
plt.show()

plt.figure(figsize=(7, 10))
plt.title(f'Spacecraft Pitch Angle vs MLT (Background)')
p_PAvMLT_BG = plt.pcolormesh(MLT_bins, PA_bins, H_PAvMLT_BG.T, vmin=0)
plt.colorbar(p_PAvMLT_BG, orientation='horizontal', label='Number of occurances')
plt.xlabel('MLT')
plt.ylabel('Pitch Angle')
plt.show()

NameError: name 'plt' is not defined